<a href="https://colab.research.google.com/github/antoniocfetngnu/InteligArtificial1/blob/main/redesNeuronales/lab5_RedesNeuronales_Pytorch_Calderon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descripcion Dataset

Este conjunto de datos resume un conjunto heterogéneo de características sobre artículos publicados por Mashable en un período de dos años. El objetivo es predecir el número de veces que se comparten en redes sociales (popularidad).
Adaptacion Salida (Clasificacion)

En este conjunto de datos, la columna "shares" se considera como la etiqueta (o salida) que se busca predecir. Para facilitar la clasificación, se ha adaptado la columna de "shares" a tres categorías de salida: 0, 1 y 2. Estas categorías corresponden a la cantidad de veces que se comparten en redes sociales, dividiendo el valor original de "shares" entre la media y luego dividiendo entre 2 para obtener tres categorías distintas.

In [ ]:
# utilizado para la manipulación de directorios y rutas
import os
# Cálculo científico y vectorial para python
import numpy as np
import pandas as pd
# Libreria para graficos
from matplotlib import pyplot as plt


import torch
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import TensorDataset,DataLoader  # Gives easier dataset managment by creating mini batches etc.

import torchvision # torch package for vision related things
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
from tqdm import tqdm  # For nice progress bar!

from scipy import optimize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

##Preparación del Dataset
Se realizó la misma preparación que se realizó con el ejercicio sin Pytorch, para poder tener una comparacion entre ambos resultados.

In [ ]:
df = pd.read_csv('OnlineNewsPopularity.csv')
pd.set_option('display.max_columns', None)
valor_maximo = df[' shares'].max()
valor_minimo = df[' shares'].min()

print(f"Valor máximo: {valor_maximo}")
print(f"Valor mínimo: {valor_minimo}")

media_shares = df[' shares'].mean()

# Divide los valores en 3 categorías basadas en la media
df['categoria_shares'] = pd.cut(df[' shares'], bins=[0, media_shares/2, media_shares, float('inf')], labels=[0, 1, 2], right=False)

# Verificar los resultados
cantidad_por_categoria = df['categoria_shares'].value_counts()
print("Cantidad por categoría actualizada:")
print(cantidad_por_categoria)


df = df.drop('url', axis=1)
df = df.dropna()
print("Categorias:\n0: Desde 0 a ",media_shares/2,"\n1: Desde ",media_shares/2," hasta ",media_shares,"\n2: Desde ",media_shares, " en adelante")
df['categoria_shares'] = df['categoria_shares'].astype(int)


# Verificar el cambio
print(df['categoria_shares'].head(10))

df = df.drop(' shares', axis=1)

print(df.describe())

Valor máximo: 843300
Valor mínimo: 1
Cantidad por categoría actualizada:
categoria_shares
0    22542
1     9023
2     8079
Name: count, dtype: int64
Categorias:
0: Desde 0 a  1697.6900918171727 
1: Desde  1697.6900918171727  hasta  3395.3801836343455 
2: Desde  3395.3801836343455  en adelante
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    2
9    0
Name: categoria_shares, dtype: int64
          timedelta   n_tokens_title   n_tokens_content   n_unique_tokens  \
count  39644.000000     39644.000000       39644.000000      39644.000000   
mean     354.530471        10.398749         546.514731          0.548216   
std      214.163767         2.114037         471.107508          3.520708   
min        8.000000         2.000000           0.000000          0.000000   
25%      164.000000         9.000000         246.000000          0.470870   
50%      339.000000        10.000000         409.000000          0.539226   
75%      542.000000        12.000000         716.000000     

In [ ]:
X = df.drop(columns=['categoria_shares'])  # Características, se eliminan las columnas de etiquetas
y = df['categoria_shares']  # Etiquetas

# Verificar las dimensiones de X y y
print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", y.shape)

print(y.value_counts())

Dimensiones de X: (39644, 59)
Dimensiones de y: (39644,)
categoria_shares
0    22542
1     9023
2     8079
Name: count, dtype: int64


##Separacion de entrenamiento y Prueba


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Verificar la proporción de clases en los conjuntos de entrenamiento y prueba
print("Proporción de clases en y_train:")
print(y_train.value_counts(normalize=True))

print("\nProporción de clases en y_test:")
print(y_test.value_counts(normalize=True))

Proporción de clases en y_train:
categoria_shares
0    0.568627
1    0.227589
2    0.203784
Name: proportion, dtype: float64

Proporción de clases en y_test:
categoria_shares
0    0.568546
1    0.227645
2    0.203809
Name: proportion, dtype: float64


In [ ]:
class RedNeuronalMLS(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RedNeuronalMLS, self).__init__()
        # Our first linear layer take input_size, in this case 784 nodes to 50
        # and our second linear layer takes 50 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        """
        x here is the mnist images and we run it through fc1, fc2 that we created above.
        we also add a ReLU activation function in between and for that (since it has no parameters)
        I recommend using nn.functional (F)
        """
        x = self.fc1(x)
        x = F.sigmoid(x)
        # x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

Hiperparámetros y sus valores escogidos tratando de alcanzar una buena precisión

In [ ]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = X_train.shape[1]
num_classes = len(np.unique(y_train))
learning_rate = 0.0001
batch_size = 10000
num_epochs = 8

cuda


###Transformacion de nuestro conjunto de datos (dataframe de pandas) en Tensores PyTorch para poder usarlos en el DataLoader

In [ ]:
# Crear Tensores PyTorch para los conjuntos de datos de entrenamiento y prueba
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Crear el conjunto de datos de entrenamiento y prueba utilizando TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Crear los DataLoader para el entrenamiento y prueba
# batch_size = 64  # Elige el tamaño del lote que desees
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

In [ ]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=59, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=3, bias=True)
)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

100%|██████████| 4/4 [00:00<00:00,  9.90it/s]


torch.Size([10000, 59])
torch.Size([10000, 59])
torch.Size([10000, 59])
torch.Size([1715, 59])


100%|██████████| 4/4 [00:00<00:00, 10.03it/s]


torch.Size([10000, 59])
torch.Size([10000, 59])
torch.Size([10000, 59])
torch.Size([1715, 59])


  0%|          | 0/4 [00:00<?, ?it/s]

...

torch.Size([10000, 59])
torch.Size([10000, 59])
torch.Size([1715, 59])


  0%|          | 0/4 [00:00<?, ?it/s]

torch.Size([10000, 59])


100%|██████████| 4/4 [00:00<00:00, 10.19it/s]

torch.Size([10000, 59])
torch.Size([10000, 59])
torch.Size([1715, 59])


In [ ]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Accuracy on training set: {p_train*100:.2f}")
print(f"Accuracy on test set: {p_test*100:.2f}")

Accuracy on training set: 56.82
Accuracy on test set: 56.79


Durante el proceso de experimentación utilizando PyTorch, se exploraron y ajustaron parámetros que influyen en la eficacia y precisión del modelo de red neuronal:

**Learning Rate **(Tasa de Aprendizaje): Este parámetro determina la magnitud de los ajustes que se aplican a los pesos del modelo durante cada iteración del entrenamiento.

**Número de Epochs **(Épocas): Las épocas representan la cantidad de veces que el modelo ve el conjunto de datos completo durante el entrenamiento.

**Batch Size** (Tamaño del Lote): Este parámetro define la cantidad de ejemplos de entrenamiento que se utilizan en cada paso de actualización de los pesos del modelo.

En conclusión, se obtuvo una precisíon un poco inferior pero similar al anterior ejercicio realizado sin Pytorch, lo que nos puede dar un indicativo que la precisión alcanzada en el anterior ejercicio no se debe a un problema con la forma en que se desarrolló el ejercicio, sin embargo eso no significa que una precision del 56-58% sea la esperada.

La utilización de PyTorch permitió un control detallado y eficaz sobre los parámetros del modelo de red neuronal.
Luego de modificar los valores de los parametros buscando una mejor precision, (mas epocas, menor/ tamaño de lotes, un coeficiente de aprendizaje mas pequeño para que tarde mas pero buscando precision)
logramos alcanzar una precision de 56% como maximo, y una de 20% como mínimo, por tanto se mantuvo los valores con los que se obtuvo mayor precisión.
